# 📒 Kaggle_AllInOne_Pro2_Enhanced.ipynb – 최고 성능 버전

## 🎯 Pro2 Enhanced 주요 개선사항

### ✅ 학습 개선
- Val Accuracy + Confusion Matrix + Classification Report
- Best 모델: Val Acc 우선 저장 + 체크포인트 재개
- 학습 곡선 시각화 (Train Loss 포함)
- 라벨 마스킹 (프롬프트 손실 제외)
- 검증 데이터 train=False
- Early Stopping 옵션

### ✅ 추론 개선
- Direct Logits (개선된 토큰 확률 계산)
- TTA [0.9, 1.0, 1.1] + 배치 추론
- pad_token_id 자동 보정
- 에러 핸들링 강화

### ✅ 앙상블 개선
- Temperature Scaling (실제 구현)
- 확률 앙상블 + Weighted Voting
- 확률 컬럼 저장

### ✅ 시스템 개선
- 강력한 에러 핸들링
- 로깅 시스템 (파일 + 콘솔)
- 메모리 최적화
- 코드 중복 제거

### ⚙️ 튜닝 설정
```python
USE_SAMPLE=False, IMAGE_SIZE=512, NUM_EPOCHS=3
GRAD_ACCUM_STEPS=8, WARMUP_RATIO=0.06, LORA_R=16
USE_DIRECT_LOGIT_DECODE=True, TTA_SCALES=[0.9,1.0,1.1]
ENSEMBLE_METHOD='prob', USE_TEMPERATURE_SCALING=True
EARLY_STOPPING_PATIENCE=2
```

**🤖 SSAFY AI Project 2025**

## 📦 1. 패키지 설치

In [ ]:
# !pip install -q transformers accelerate peft bitsandbytes datasets pillow pandas torch torchvision scikit-learn matplotlib seaborn tqdm scipy --upgrade
# !pip install -q qwen-vl-utils==0.0.8
print("✅ 설치 완료! 런타임 재시작하세요.")

## 📚 2. 라이브러리 임포트

In [ ]:
import os, sys, re, math, random, warnings, json, pickle, logging
from datetime import datetime
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Any, Optional, Tuple
from collections import Counter, defaultdict
import unicodedata
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.swa_utils import AveragedModel, SWALR

from transformers import (
    AutoModelForVision2Seq,
    Qwen2_5_VLForConditionalGeneration,
    AutoProcessor,
    BitsAndBytesConfig,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from qwen_vl_utils import process_vision_info

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.optimize import minimize

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')
Image.MAX_IMAGE_PIXELS = None
sns.set_style('whitegrid')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
print(f"   PyTorch: {torch.__version__}")

## ⚙️ 3. Config 설정 (Pro2 Enhanced)

In [ ]:
class Config:
    # 시드
    SEED = 42
    
    # 모델
    MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
    IMAGE_SIZE = 512
    USE_ADVANCED_MODEL = False
    
    # 데이터
    DATA_DIR = "/content"
    TRAIN_CSV = f"{DATA_DIR}/train.csv"
    TEST_CSV = f"{DATA_DIR}/test.csv"
    
    # K-Fold
    N_FOLDS = 3
    USE_KFOLD = True
    TRAIN_FOLDS = [0, 1, 2]
    
    # QLoRA
    LORA_R = 16
    LORA_ALPHA = 32
    LORA_DROPOUT = 0.05
    TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    
    # 학습
    NUM_EPOCHS = 3
    BATCH_SIZE = 1
    GRAD_ACCUM_STEPS = 8
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 0.01
    WARMUP_RATIO = 0.06
    MAX_GRAD_NORM = 1.0
    
    # Early Stopping
    USE_EARLY_STOPPING = False
    EARLY_STOPPING_PATIENCE = 2
    
    # 고급 기법
    USE_AMP = True
    USE_EMA = True
    EMA_DECAY = 0.999
    USE_SWA = True
    SWA_START_EPOCH = 1
    USE_COSINE_SCHEDULE = True
    
    # TTA
    USE_TTA = True
    TTA_SCALES = [0.9, 1.0, 1.1]
    
    # 추론
    USE_DIRECT_LOGIT_DECODE = True
    USE_BATCH_INFERENCE = True  # Enhanced: 배치 추론 활성화
    INFER_BATCH_SIZE = 4
    MAX_NEW_TOKENS = 8
    
    # Temperature Scaling
    USE_TEMPERATURE_SCALING = True
    
    # 앙상블
    ENSEMBLE_METHOD = "prob"  # "prob" or "vote" or "weighted"
    FOLD_WEIGHTS = None  # None이면 동일 가중치, [0.4, 0.3, 0.3] 등 가능
    
    # 저장
    SAVE_DIR = f"{DATA_DIR}/checkpoints"
    OUTPUT_DIR = f"{DATA_DIR}/outputs"
    LOG_DIR = f"{DATA_DIR}/logs"
    SAVE_EVERY_EPOCH = True  # 매 에폭마다 체크포인트 저장
    
    # 샘플링
    USE_SAMPLE = False
    SAMPLE_SIZE = 200
    
    # 프롬프트
    SYSTEM_INSTRUCT = (
        "You are a helpful visual question answering assistant. "
        "Answer using exactly one letter among a, b, c, or d. No explanation."
    )
    
    # 로깅
    LOG_LEVEL = logging.INFO
    LOG_TO_FILE = True

cfg = Config()

# 디렉토리 생성
for dir_path in [cfg.SAVE_DIR, cfg.OUTPUT_DIR, cfg.LOG_DIR]:
    Path(dir_path).mkdir(parents=True, exist_ok=True)

# 로깅 설정
def setup_logging():
    logger = logging.getLogger('VQA')
    logger.setLevel(cfg.LOG_LEVEL)
    logger.handlers.clear()
    
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
    # 콘솔 핸들러
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    
    # 파일 핸들러
    if cfg.LOG_TO_FILE:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        file_handler = logging.FileHandler(f"{cfg.LOG_DIR}/training_{timestamp}.log")
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    
    return logger

logger = setup_logging()

# 시드 고정
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.SEED)
logger.info(f"✅ Config 설정 완료")
logger.info(f"   Model: {cfg.MODEL_ID}")
logger.info(f"   Image Size: {cfg.IMAGE_SIZE}")
logger.info(f"   Epochs: {cfg.NUM_EPOCHS}, Grad Accum: {cfg.GRAD_ACCUM_STEPS}")
logger.info(f"   LoRA R: {cfg.LORA_R}, Warmup: {cfg.WARMUP_RATIO}")
logger.info(f"   Direct Logits: {cfg.USE_DIRECT_LOGIT_DECODE}, TTA: {cfg.USE_TTA}")
logger.info(f"   Ensemble: {cfg.ENSEMBLE_METHOD}, Temp Scaling: {cfg.USE_TEMPERATURE_SCALING}")
print(f"\n📝 로그 저장 위치: {cfg.LOG_DIR}")

## 📊 4. 데이터 로드 & EDA

In [ ]:
try:
    train_df = pd.read_csv(cfg.TRAIN_CSV)
    test_df = pd.read_csv(cfg.TEST_CSV)
    logger.info(f"📁 Train: {len(train_df):,} samples")
    logger.info(f"📁 Test: {len(test_df):,} samples")
except Exception as e:
    logger.error(f"❌ 데이터 로드 실패: {e}")
    raise

# 데이터 검증
required_cols = ['question', 'a', 'b', 'c', 'd', 'answer']
missing_cols = set(required_cols) - set(train_df.columns)
if missing_cols:
    raise ValueError(f"❌ 필수 컬럼 누락: {missing_cols}")

# 이미지 경로 컬럼 확인
img_col = 'path' if 'path' in train_df.columns else 'image'
logger.info(f"📷 이미지 컬럼: {img_col}")

if cfg.USE_SAMPLE:
    train_df = train_df.sample(n=min(cfg.SAMPLE_SIZE, len(train_df)), random_state=cfg.SEED).reset_index(drop=True)
    logger.warning(f"⚠️  Sampled {len(train_df)} samples")

logger.info(f"\n📊 Answer Distribution:")
answer_dist = train_df['answer'].value_counts().sort_index()
for ans, count in answer_dist.items():
    logger.info(f"   {ans}: {count:4d} ({count/len(train_df)*100:.1f}%)")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
answer_dist.plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Answer Distribution (Train)', fontsize=12, weight='bold')
axes[0].set_xlabel('Answer')
axes[0].set_ylabel('Count')
axes[0].grid(axis='y', alpha=0.3)

train_df['question_len'] = train_df['question'].str.len()
train_df['question_len'].hist(bins=30, ax=axes[1], color='salmon')
axes[1].set_title('Question Length Distribution', fontsize=12, weight='bold')
axes[1].set_xlabel('Length (chars)')
axes[1].set_ylabel('Count')
axes[1].grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(f"{cfg.LOG_DIR}/data_distribution.png", dpi=150, bbox_inches='tight')
plt.show()

logger.info(f"✅ 데이터 로드 및 EDA 완료")

## 🔄 5. Stratified K-Fold CV

In [ ]:
if cfg.USE_KFOLD:
    skf = StratifiedKFold(n_splits=cfg.N_FOLDS, shuffle=True, random_state=cfg.SEED)
    train_df['fold'] = -1
    for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['answer'])):
        train_df.loc[val_idx, 'fold'] = fold
    
    logger.info(f"✅ {cfg.N_FOLDS}-Fold CV 생성")
    fold_dist = train_df['fold'].value_counts().sort_index()
    for fold, count in fold_dist.items():
        logger.info(f"   Fold {fold}: {count:4d} samples")
    
    # Fold별 답변 분포 확인
    logger.info(f"\n📊 Answer Distribution per Fold:")
    for fold in range(cfg.N_FOLDS):
        fold_data = train_df[train_df['fold'] == fold]
        dist = fold_data['answer'].value_counts(normalize=True).sort_index()
        dist_str = ", ".join([f"{k}:{v:.2%}" for k, v in dist.items()])
        logger.info(f"   Fold {fold}: {dist_str}")
else:
    split_idx = int(len(train_df) * 0.9)
    train_df['fold'] = -1
    train_df.loc[split_idx:, 'fold'] = 0
    logger.info(f"✅ Single split (90:10)")

## 🗂️ 6. Dataset & DataCollator (Enhanced)

✅ **개선사항**:
- 에러 핸들링 강화
- 이미지 로드 실패 시 fallback
- 라벨 마스킹 정교화

In [ ]:
def build_mc_prompt(question, a, b, c, d):
    """Multiple Choice 프롬프트 생성"""
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
    )

class VQADataset(Dataset):
    """Enhanced VQA Dataset with Error Handling"""
    
    def __init__(self, df, processor, data_dir="", train=True, use_advanced=False, img_col='path'):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.data_dir = data_dir
        self.train = train
        self.use_advanced = use_advanced
        self.img_col = img_col
        self.failed_images = []
    
    def __len__(self):
        return len(self.df)
    
    def _load_image(self, img_path):
        """안전한 이미지 로드 with fallback"""
        try:
            img = Image.open(img_path).convert("RGB")
            # 이미지 검증
            if img.size[0] < 10 or img.size[1] < 10:
                raise ValueError(f"이미지 크기가 너무 작음: {img.size}")
            return img, True
        except Exception as e:
            if len(self.failed_images) < 10:  # 처음 10개만 로그
                logger.warning(f"⚠️  이미지 로드 실패 ({img_path}): {e}")
            self.failed_images.append(img_path)
            # Fallback: 흰색 이미지
            return Image.new('RGB', (cfg.IMAGE_SIZE, cfg.IMAGE_SIZE), color='white'), False
    
    def __getitem__(self, idx):
        try:
            row = self.df.iloc[idx]
            
            # 이미지 로드
            img_path = os.path.join(self.data_dir, row[self.img_col])
            img, success = self._load_image(img_path)
            
            # 프롬프트 생성
            user_text = build_mc_prompt(
                str(row["question"]), str(row["a"]), 
                str(row["b"]), str(row["c"]), str(row["d"])
            )
            
            messages = [
                {"role": "system", "content": [{"type": "text", "text": cfg.SYSTEM_INSTRUCT}]},
                {"role": "user", "content": [
                    {"type": "image", "image": img},
                    {"type": "text", "text": user_text}
                ]}
            ]
            
            # 학습 시에만 정답 포함
            answer = None
            if self.train:
                answer = str(row["answer"]).strip().lower()
                if answer not in ['a', 'b', 'c', 'd']:
                    logger.warning(f"⚠️  잘못된 답변 ({idx}): {answer}, 'a'로 대체")
                    answer = 'a'
                messages.append({
                    "role": "assistant",
                    "content": [{"type": "text", "text": answer}]
                })
            
            return {"messages": messages, "image": img, "answer": answer, "idx": idx}
        
        except Exception as e:
            logger.error(f"❌ Dataset __getitem__ 에러 (idx={idx}): {e}")
            # Fallback: 더미 데이터
            dummy_img = Image.new('RGB', (cfg.IMAGE_SIZE, cfg.IMAGE_SIZE), color='white')
            messages = [
                {"role": "system", "content": [{"type": "text", "text": cfg.SYSTEM_INSTRUCT}]},
                {"role": "user", "content": [
                    {"type": "image", "image": dummy_img},
                    {"type": "text", "text": "dummy question\n(a) a\n(b) b\n(c) c\n(d) d"}
                ]}
            ]
            if self.train:
                messages.append({"role": "assistant", "content": [{"type": "text", "text": "a"}]})
            return {"messages": messages, "image": dummy_img, "answer": 'a' if self.train else None, "idx": idx}

@dataclass
class DataCollator:
    """Enhanced Data Collator"""
    processor: Any
    train: bool = True
    use_advanced: bool = False
    
    def __call__(self, batch):
        texts, images, answers = [], [], []
        
        for sample in batch:
            try:
                text = self.processor.apply_chat_template(
                    sample["messages"],
                    tokenize=False,
                    add_generation_prompt=False
                )
                text = unicodedata.normalize('NFKC', text)
                texts.append(text)
                images.append(sample["image"])
                answers.append(sample["answer"])
            except Exception as e:
                logger.error(f"❌ DataCollator 에러: {e}")
                continue
        
        if not texts:
            raise ValueError("❌ 배치에 유효한 데이터가 없습니다")
        
        # 인코딩
        enc = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt"
        )
        
        # 라벨 마스킹
        if self.train:
            labels = enc["input_ids"].clone()
            for i, answer in enumerate(answers):
                if answer is None or answer not in ['a', 'b', 'c', 'd']:
                    labels[i, :] = -100
                else:
                    labels[i, :] = -100
                    answer_ids = self.processor.tokenizer.encode(answer, add_special_tokens=False)
                    if len(answer_ids) > 0:
                        labels[i, -len(answer_ids):] = torch.tensor(answer_ids)
            enc["labels"] = labels
        
        return enc

logger.info("✅ Dataset & DataCollator 정의 완료")

## 🤖 7. Model & Processor 로드 (Enhanced)

✅ **개선사항**:
- 에러 핸들링
- 모델 로드 실패 시 재시도
- 메모리 체크

In [ ]:
def create_model_and_processor(model_id, use_advanced=False, max_retries=3):
    """Enhanced 모델 및 Processor 생성"""
    
    for attempt in range(max_retries):
        try:
            logger.info(f"🔧 모델 로드 시도 {attempt+1}/{max_retries}...")
            
            # 양자화 설정
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
            )
            
            # Processor 로드
            processor = AutoProcessor.from_pretrained(
                model_id,
                min_pixels=cfg.IMAGE_SIZE * cfg.IMAGE_SIZE,
                max_pixels=cfg.IMAGE_SIZE * cfg.IMAGE_SIZE,
                trust_remote_code=True,
            )
            logger.info("✅ Processor 로드 완료")
            
            # 모델 로드
            if use_advanced:
                base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
                    model_id,
                    quantization_config=bnb_config,
                    trust_remote_code=True,
                    torch_dtype=torch.float16,
                    attn_implementation="sdpa",
                )
            else:
                base_model = AutoModelForVision2Seq.from_pretrained(
                    model_id,
                    quantization_config=bnb_config,
                    trust_remote_code=True,
                )
            logger.info("✅ Base model 로드 완료")
            
            # QLoRA 준비
            base_model = prepare_model_for_kbit_training(base_model)
            base_model.gradient_checkpointing_enable()
            
            # LoRA Config
            lora_config = LoraConfig(
                r=cfg.LORA_R,
                lora_alpha=cfg.LORA_ALPHA,
                lora_dropout=cfg.LORA_DROPOUT,
                bias="none",
                target_modules=cfg.TARGET_MODULES,
                task_type="CAUSAL_LM",
            )
            
            # PEFT 모델 생성
            model = get_peft_model(base_model, lora_config)
            model.print_trainable_parameters()
            
            # 디바이스로 이동
            model = model.to(device)
            
            # 메모리 체크
            if torch.cuda.is_available():
                allocated = torch.cuda.memory_allocated() / 1e9
                reserved = torch.cuda.memory_reserved() / 1e9
                logger.info(f"💾 GPU Memory - Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB")
            
            logger.info("✅ 모델 로드 완료")
            return model, processor
            
        except Exception as e:
            logger.error(f"❌ 모델 로드 실패 (시도 {attempt+1}): {e}")
            if attempt < max_retries - 1:
                logger.info("🔄 재시도 중...")
                torch.cuda.empty_cache()
                import time
                time.sleep(5)
            else:
                raise

model, processor = create_model_and_processor(cfg.MODEL_ID, cfg.USE_ADVANCED_MODEL)

# 이미지 컬럼 자동 감지
img_col = 'path' if 'path' in train_df.columns else 'image'
logger.info(f"📷 이미지 컬럼: {img_col}")